In [1]:
%load_ext autoreload
%autoreload 2

from IPython.display import display, HTML
import sys
sys.path.append("./../src")

cache = '2022-05-13'

In [2]:
import spacy
from os import path
from torch.utils.data import DataLoader

spacy_model = spacy.load('en_core_web_md')
cache_path = path.join('..','src','_out')
vector_path = path.join(cache_path, 'models', '.vector_cache')
dataset_path = path.join(cache_path, 'dataset')
vocab_path = path.join(dataset_path, 'esnli', 'vocab_lemma-lower.pt')
nb_data = 128

In [3]:
from torch.nn import Module


class PaddingToken(Module):
    
    def __init__(self, pad_value):
        super(PaddingToken, self).__init__()
        self.pad_value = pad_value
        
    def forward(self, text):
        txt_lens = [len(t) for t in text]
        max_len = max(txt_lens)
        txt_lens = [max_len - l for l in txt_lens]
        for i in range(len(text)):
            text[i] += [self.pad_value] * txt_lens[i]
        return text

In [7]:
from data.transforms import LemmaLowerTokenizerTransform, SpacyTokenizerTransform
from data.esnli.transforms import HeuristicTransform, HighlightTransform, GoldLabelTransform
from torchtext.vocab.vectors import GloVe
from supervision_heuristic import LitData
import torchtext.transforms as T
import torch
from data.esnli.dataset import ESNLIDataset

vocab = torch.load(vocab_path)

heuristic_transform = HeuristicTransform(
    vectors=GloVe(cache=path.join(cache_path, 'models', '.vector_cache')),
    spacy_model=spacy_model,
    normalize='softmax'
)

tokenizer = LemmaLowerTokenizerTransform(spacy_model)

text_transform = T.Sequential(
    tokenizer,
    T.VocabTransform(vocab),
    T.ToTensor(padding_value=vocab['[pad]'])
)

token_transform = T.Sequential(
    SpacyTokenizerTransform(spacy_model),
    PaddingToken(pad_value='[pad]')
)

gt_transform = T.Sequential(
    tokenizer,
    HighlightTransform(),
    T.ToTensor(padding_value=False)
)

class_transform = T.Sequential(
    GoldLabelTransform(),
    T.ToTensor()
)

esnli = ESNLIDataset(root=dataset_path, split='test', n_data=nb_data)
esnli.data = esnli.data[esnli.data['label'] != 'neutral'].reset_index()

def list2dict(batch):
    # convert list of dict to dict of list

    if isinstance(batch, dict): return {k: list(v) for k, v in batch.items()} # handle case where no batch
    return {k: [row[k] for row in batch] for k in batch[0]}

def collate(batch):
    # prepare batch of data for dataloader
    batch = list2dict(batch)
    heuristics = heuristic_transform(premise=batch['premise'], hypothesis=batch['hypothesis'])

    features = {
        'premise': text_transform(batch['premise']),
        'hypothesis': text_transform(batch['hypothesis']),
        'heuristic_premise': heuristics['premise'],
        'heuristic_hypothesis': heuristics['hypothesis'],
        'groundtruth_premise': gt_transform(batch['highlight_premise']),
        'groundtruth_hypothesis': gt_transform(batch['highlight_hypothesis']),
        'tokens_premise': token_transform(batch['premise']),
        'tokens_hypothesis': token_transform(batch['hypothesis']),
        'label': batch['label']
    }
    features['padding_mask_premise'] = features['premise'] == vocab['[pad]']
    features['padding_mask_hypothesis'] = features['hypothesis'] == vocab['[pad]']
    y = class_transform(batch['label'])
    return features, y

test_iter = DataLoader(esnli, batch_size=50, shuffle=False, collate_fn=collate)

ModuleNotFoundError: No module named 'supervision_heuristic'

In [ ]:
import string 

def html_highlight(tokens, mask, padding_filter=None):
    assert len(tokens) == len(mask), f'Incompatible length: {len(tokens)}(tokens) vs {len(mask)}(mask)'
    return ' '.join([f'<span style="background-color:rgba(135,206,250,{0.8*m});">{word}</span>' for word, m in zip(tokens, mask) if padding_filter is None or padding_filter != word]) 


def visualize_highlight(tokens: list, mask: list, padding_filter=None):
    assert len(tokens) == len(mask), f'Unequal length \ntokens ({len(tokens)}) vs mask ({len(mask)})'
    
    nb_line = len(tokens)
    visual = [None] * nb_line
    for line in range(nb_line):
        visual[line] = html_highlight(tokens[line], mask[line], padding_filter=padding_filter)
    return visual

def visualize_pair(p_tokens, h_tokens, p_attention, h_attention, max_line=-1, padding_filter=None):
    html = '<table style="font-size:120%;border-collapse: collapse;" cellpadding=0 cellspacing=0>'
    html += '<tr> <th>Premise</th> <th>Hypothesis</th> </tr>'
    for i in range(len(p_attention)):
        html += '<tr> <td>' +  html_highlight(p_tokens[i], p_attention[i], padding_filter) + '</td><td>' + html_highlight(h_tokens[i], h_attention[i], padding_filter) +'</td></tr>'
        if max_line > 0 and i > max_line: break
    html += '</table>'
    return html

def rescale(attention):
    rescale_attention = None
    
    # Check torch version
    if torch.is_tensor(attention):
        v_max = torch.max(attention, dim=1, keepdim=True).values
        v_min = torch.min(attention, dim=1, keepdim=True).values
        v_min[v_min == v_max] = 0.
        rescale_attention = (attention - v_min)/(v_max - v_min)
    
    # Check numpy version
    elif isinstance(attention, (np.ndarray, np.generic)):
        v_max = np.max(attention, axis=1)
        v_min = np.min(attention, axis=1)
        v_min[v_min == v_max] = 0.
        rescale_attention = (attention - v_min)/(v_max - v_min)
    
    else:
        raise NotImplementedError
            
    
    return rescale_attention

In [ ]:
x, y_true = next(iter(test_iter))

h_premise = x['heuristic_premise']
h_hypothesis = x['heuristic_hypothesis']
tokens_premise = x['tokens_premise']
tokens_hypothesis = x['tokens_hypothesis']

h_premise = rescale(h_premise)
h_hypothesis = rescale(h_hypothesis)

display(HTML('<h3>Heuristic</h3>'))
display(HTML(visualize_pair(tokens_premise, tokens_hypothesis, h_premise, h_hypothesis, padding_filter='[pad]')))

In [61]:
with open('heuristic.html', 'w') as f:
    f.write(visualize_pair(tokens_premise, tokens_hypothesis, h_premise, h_hypothesis, padding_filter='[pad]'))

## Qualification visuelle

Support pour la visualisation

### Régularisation par l'entropie

In [48]:
def html_per_row(model_name:str, p_tokens, p_attention, h_tokens, h_attention, padding_filter=None, label = None):
    html = '<tr>'
    html += '<td><b>'+ model_name +'</b></td>' 
    html += '<td>' +  html_highlight(p_tokens, p_attention, padding_filter) + '</td>'
    html += '<td>' + html_highlight(h_tokens, h_attention, padding_filter) +'</td>'
    if label is not None:
        html += '<td>' + str(label) + '</td>'
    html += '</tr>'
    return html

def html_attention_dict(model_attentions:dict, label=None, padding_filter=None):
    html = '<table style="font-size:200%">'
    html += '<tr> <th></th> <th>Premise</th> <th>Hypothesis</th> '
    if label is not None: html += '<th>Label</th>'
    html += ' </tr>'
    
    tokens_p, tokens_h = model_attentions['tokens']
    
    for model_name, heatmap in model_attentions.items():
        if model_name == 'tokens': continue
        hm_p, hm_h = heatmap
        html += html_per_row(model_name, tokens_p, hm_p, tokens_h, hm_h, padding_filter=padding_filter, label=label)
        
    html += '</table>'
    return html

In [ ]:
from torch import nn
from modules.loss import IoU
from supervision_delta_sigmoid import LitModel
import os

def html_render(model_outputs):
        html = ''
        table_len = len(model_outputs['GROUNDTRUTH']['Label'])
        for i in range(table_len):
            html += '<table style="font-size:150%;border-collapse:collapse;border:solid black;">'
            
            html += '<thead>'
            html += '<tr style="border:solid black;"><th></th>' # One xtra head for model's name
            for column_name in model_outputs['GROUNDTRUTH'].keys():
                html+= '<th style="border-style:solid black;">'+ column_name +'</th>'
            html += ' </tr>'
            html += '</thead>'
            
            html += '<tbody>'
            for name, model_content in model_outputs.items():
                html += '<tr>'
                html += '<td style="border:solid black;" align="right"><b>' + name + '</b></td>'

                for k, output in model_content.items():
                    displ = output[i] if output is not None else 'N/A'
                    if isinstance(displ, float):
                        displ = str(round(displ, 3))
                    html += '<td style="border:solid black;" align="justify">' + displ + '</td>'

                html += '</tr>'
                
            html += '</tbody>'
            html += '</table>'
        return html

path_name_dict = {
    'Baseline': '/Users/dunguyen/Projects/IGRIDA/historic/2022-05-22/regularization_entropy/lambda=0',
    '&lambda;=0.01': '/Users/dunguyen/Projects/IGRIDA/historic/2022-05-22/regularization_entropy/lambda=0.001',   
    #'&lambda;=0.03': '/Users/dunguyen/Projects/IGRIDA/historic/2022-05-22/regularization_entropy/lambda=0.003',   
    '&lambda;=0.02': '/Users/dunguyen/Projects/IGRIDA/historic/2022-05-22/regularization_entropy/lambda=0.007',   
    '&lambda;=0.06': '/Users/dunguyen/Projects/IGRIDA/historic/2022-05-22/regularization_entropy/lambda=0.01',   
}

cache_path='/Users/dunguyen/Projects/IGRIDA/historic/2022-05-22/cache'
vocab = torch.load(cache_path+'/dataset/esnli/vocab_lemma-lower.pt')
loss_fn = nn.CrossEntropyLoss()
reg_loss_fn = IoU()

# x, y_true = next(iter(test_iter))

display(HTML('<h3>Impact of Regularization on attention map </h3>'))

for id_batch, batch in enumerate(test_iter):
    
    # if id_batch > 5: break
    
    x, y_true = batch

    h_premise = x['heuristic_premise']
    h_hypothesis = x['heuristic_hypothesis']
    tokens_premise = x['tokens_premise']
    tokens_hypothesis = x['tokens_hypothesis']
    groundtruth_premise = x['groundtruth_premise']
    groundtruth_hypothesis = x['groundtruth_hypothesis']
    padding_mask = [x['padding_mask_premise'], x['padding_mask_hypothesis']]
    labels = x['label']

    h_premise = rescale(h_premise)
    h_hypothesis = rescale(h_hypothesis)


    # For each model, we save all of its predictions
    model_outputs = {}

    penalty = ['_'] * len(y_true)

    # baseline
    model_outputs['GROUNDTRUTH'] = {
        'Premise': [html_highlight(tokens, gt, padding_filter='[pad]') for tokens, gt in zip(tokens_premise, groundtruth_premise)],
        'Hypothesis': [html_highlight(tokens, gt, padding_filter='[pad]') for tokens, gt in zip(tokens_hypothesis, groundtruth_hypothesis)],
        'Label': labels,
        #'Loss': None,
        #'Attention IoU': penalty
    }

    # effect of different models
    for name, model_path in path_name_dict.items():

        model_outpath = model_path + '/.out'
        model = LitModel.load_from_checkpoint(model_path + '/checkpoints/best.ckpt', cache_path=cache_path+'models', mode='dev', vocab=vocab, reg_loss='iou')
        model.eval()

        # caching output
        y_hat, (att_premise, att_hypothesis) = model(x)

        flat_a_hat, flat_a_true = model.flatten_attention(
            a_hat=(att_premise, att_hypothesis),
            a_true=(groundtruth_premise, groundtruth_hypothesis),
            y_true=y_true,
            pad_mask=padding_mask,
            normalize='sigmoid'
        )

        #loss = [loss_fn(y_hat[i], y_true[i]).item() for i in range(len(y_hat))]
        if flat_a_true is None:
            penalty = ['N/A'] * len(y_hat)
        else:
            penalty = reg_loss_fn(flat_a_hat, flat_a_true)
            penalty = [penalty.item()] * len(y_hat)

        
        att_premise_rescale = rescale(torch.softmax(att_premise, axis=1))
        att_hypothesis_rescale = rescale(torch.softmax(att_hypothesis, axis=1))
            
        model_outputs[name] =  {
            'Premise': [html_highlight(tokens, attentions, padding_filter='[pad]') for tokens, attentions in zip(tokens_premise, att_premise_rescale)],
            'Hypothesis': [html_highlight(tokens, attentions, padding_filter='[pad]') for tokens, attentions in zip(tokens_hypothesis, att_hypothesis_rescale)],
            'Label': labels,
            # 'Loss': loss,
            #'Attention IoU': penalty
        }    

    display(HTML(html_render(model_outputs)))
    break

In [92]:
with open('regularization.html', 'w') as f:
    f.write(html_render(model_outputs))

### Attention supervision with IoU

In [ ]:
from torch import nn
from modules.loss import IoU
from supervision_delta_sigmoid import LitModel
import os

path_name_dict = {
    'Baseline': '/Users/dunguyen/Projects/IGRIDA/historic/2022-05-22/supervision_iou/lambda=0',
    '&lambda;=0.03': '/Users/dunguyen/Projects/IGRIDA/historic/2022-05-22/supervision_iou/lambda=0.003',   
    '&lambda;=0.05': '/Users/dunguyen/Projects/IGRIDA/historic/2022-05-22/supervision_iou/lambda=0.005',   
    #'&lambda;=0.008': '/Users/dunguyen/Projects/IGRIDA/historic/2022-05-22/supervision_iou/lambda=0.09',   
    '&lambda;=0.1': '/Users/dunguyen/Projects/IGRIDA/historic/2022-05-22/supervision_iou/lambda=0.5',   
    #'&lambda;=0.5': '/Users/dunguyen/Projects/IGRIDA/historic/2022-05-22/supervision_iou/lambda=0.5',   
    #'&lambda;=0.9': '/Users/dunguyen/Projects/IGRIDA/historic/2022-05-22/supervision_iou/lambda=0.9',   
}

cache_path='/Users/dunguyen/Projects/IGRIDA/historic/2022-05-22/cache'
vocab = torch.load(cache_path+'/dataset/esnli/vocab_lemma-lower.pt')
loss_fn = nn.CrossEntropyLoss()
reg_loss_fn = IoU()


display(HTML('<h3>Impact of Superivison on attention map </h3>'))

for id_batch, batch in enumerate(test_iter):
    
    # if id_batch > 5: break
    
    x, y_true = batch

    h_premise = x['heuristic_premise']
    h_hypothesis = x['heuristic_hypothesis']
    tokens_premise = x['tokens_premise']
    tokens_hypothesis = x['tokens_hypothesis']
    groundtruth_premise = x['groundtruth_premise']
    groundtruth_hypothesis = x['groundtruth_hypothesis']
    padding_mask = [x['padding_mask_premise'], x['padding_mask_hypothesis']]
    labels = x['label']

    h_premise = rescale(h_premise)
    h_hypothesis = rescale(h_hypothesis)


    # For each model, we save all of its predictions
    model_outputs = {}

    penalty = ['_'] * len(y_true)

    # baseline
    model_outputs['GROUNDTRUTH'] = {
        'Premise': [html_highlight(tokens, gt, padding_filter='[pad]') for tokens, gt in zip(tokens_premise, groundtruth_premise)],
        'Hypothesis': [html_highlight(tokens, gt, padding_filter='[pad]') for tokens, gt in zip(tokens_hypothesis, groundtruth_hypothesis)],
        'Label': labels,
        #'Loss': None,
        #'Attention IoU': penalty
    }

    # effect of different models
    for name, model_path in path_name_dict.items():

        model_outpath = model_path + '/.out'
        model = LitModel.load_from_checkpoint(model_path + '/checkpoints/best.ckpt', cache_path=cache_path+'models', mode='dev', vocab=vocab, reg_loss='iou')
        model.eval()

        # caching output
        y_hat, (att_premise, att_hypothesis) = model(x)

        flat_a_hat, flat_a_true = model.flatten_attention(
            a_hat=(att_premise, att_hypothesis),
            a_true=(groundtruth_premise, groundtruth_hypothesis),
            y_true=y_true,
            pad_mask=padding_mask,
            normalize='sigmoid'
        )

        #loss = [loss_fn(y_hat[i], y_true[i]).item() for i in range(len(y_hat))]
        if flat_a_true is None:
            penalty = ['N/A'] * len(y_hat)
        else:
            penalty = reg_loss_fn(flat_a_hat, flat_a_true)
            penalty = [penalty.item()] * len(y_hat)

        
        att_premise_rescale = rescale(torch.softmax(att_premise, axis=1))
        att_hypothesis_rescale = rescale(torch.softmax(att_hypothesis, axis=1))
            
        model_outputs[name] =  {
            'Premise': [html_highlight(tokens, attentions, padding_filter='[pad]') for tokens, attentions in zip(tokens_premise, att_premise_rescale)],
            'Hypothesis': [html_highlight(tokens, attentions, padding_filter='[pad]') for tokens, attentions in zip(tokens_hypothesis, att_hypothesis_rescale)],
            'Label': labels,
            # 'Loss': loss,
            #'Attention IoU': penalty
        }
    
    display(HTML(html_render(model_outputs)))
    break

In [94]:
with open('supervision.html', 'w') as f:
    f.write(html_render(model_outputs))

### Semi-supervised attention

In [104]:
from torch import nn
from modules.loss import IoU
from supervision_heuristic import LitModel, loss_heuristic
import os

cache_path='/Users/dunguyen/Devle/IGRIDA/historic/2022-05-22/cache/'

heuristic_transform_heuris = HeuristicTransform(
    vectors=GloVe(cache=path.join(cache_path, 'models', '.vector_cache')),
    spacy_model=tokenizer.sm,
    normalize='log_softmax'
)

def collate_heuris(batch):
    # prepare batch of data for dataloader
    batch = list2dict(batch)
    heuristics = heuristic_transform_heuris(premise=batch['premise'], hypothesis=batch['hypothesis'])

    features = {
        'premise': text_transform(batch['premise']),
        'hypothesis': text_transform(batch['hypothesis']),
        'heuristic_premise': heuristics['premise'],
        'heuristic_hypothesis': heuristics['hypothesis'],
        'groundtruth_premise': gt_transform(batch['highlight_premise']),
        'groundtruth_hypothesis': gt_transform(batch['highlight_hypothesis']),
        'tokens_premise': token_transform(batch['premise']),
        'tokens_hypothesis': token_transform(batch['hypothesis']),
        'label': batch['label']
    }
    features['padding_mask_premise'] = features['premise'] == vocab['[pad]']
    features['padding_mask_hypothesis'] = features['hypothesis'] == vocab['[pad]']
    y = class_transform(batch['label'])
    return features, y
    
heuris_test_iter = DataLoader(esnli, batch_size=50, shuffle=False, collate_fn=collate_heuris)

path_name_dict = {
    'Baseline': '/Users/dunguyen/Projects/IGRIDA/historic/2022-05-22/supervision_heuristic/lambda=0',
    '&lambda;=0.01': '/Users/dunguyen/Projects/IGRIDA/historic/2022-05-22/supervision_heuristic/lambda=0.01',    
    '&lambda;=0.03': '/Users/dunguyen/Projects/IGRIDA/historic/2022-05-22/supervision_heuristic/lambda=0.03',   
    '&lambda;=0.04': '/Users/dunguyen/Projects/IGRIDA/historic/2022-05-22/supervision_heuristic/lambda=0.04',   
}


vocab = torch.load(cache_path+'dataset/esnli/vocab_lemma-lower.pt')
loss_fn = nn.CrossEntropyLoss()
heuris_loss_fn = nn.KLDivLoss(reduction='batchmean', log_target=True)

x, y_true = next(iter(test_iter))

display(HTML('<h3>Impact of Superivison on attention map </h3>'))

for id_batch, batch in enumerate(heuris_test_iter):
    
    
    x, y_true = batch

    h_premise = x['heuristic_premise']
    h_hypothesis = x['heuristic_hypothesis']
    tokens_premise = x['tokens_premise']
    tokens_hypothesis = x['tokens_hypothesis']
    groundtruth_premise = x['groundtruth_premise']
    groundtruth_hypothesis = x['groundtruth_hypothesis']
    padding_mask = [x['padding_mask_premise'], x['padding_mask_hypothesis']]
    labels = x['label']

    # For each model, we save all of its predictions
    model_outputs = {}

    penalty = ['_'] * len(y_true)

    # baseline
    model_outputs['GROUNDTRUTH'] = {
        'Premise': [html_highlight(tokens, gt, padding_filter='[pad]') for tokens, gt in zip(tokens_premise, groundtruth_premise)],
        'Hypothesis': [html_highlight(tokens, gt, padding_filter='[pad]') for tokens, gt in zip(tokens_hypothesis, groundtruth_hypothesis)],
        'Label': labels,
    #    'Heuristic loss': penalty
    }
    
    visual_h_premise = rescale(h_premise)
    visual_h_hypothesis = rescale(h_hypothesis)
    model_outputs['HEURISTIC'] = {
        'Premise': [html_highlight(tokens, gt, padding_filter='[pad]') for tokens, gt in zip(tokens_premise, visual_h_premise)],
        'Hypothesis': [html_highlight(tokens, gt, padding_filter='[pad]') for tokens, gt in zip(tokens_hypothesis, visual_h_hypothesis)],
        'Label': labels,
    #    'Heuristic loss': penalty
    }

    # effect of different models
    for name, model_path in path_name_dict.items():
        model_outpath = model_path + '/.out'
        model = LitModel.load_from_checkpoint(model_path + '/checkpoints/best.ckpt', cache_path=cache_path+'models', mode='dev', vocab=vocab, reg_loss='iou')
        model.eval()

        # caching output
        y_hat, (att_premise, att_hypothesis) = model(x)

        B = len(att_premise)
        penalty = [loss_heuristic(heuris_loss_fn, att_premise[i].unsqueeze(0), h_premise[i].unsqueeze(0), padding_mask[0]) + loss_heuristic(heuris_loss_fn, att_hypothesis[i].unsqueeze(0), h_hypothesis[i].unsqueeze(0), padding_mask[1]) for i in range(B)]
        penalty = [p.item() for p in penalty]
        #penalty = heuris_loss_fn(att_premise, h_premise)
        
        att_premise_rescale = rescale(torch.softmax(att_premise, axis=1))
        att_hypothesis_rescale = rescale(torch.softmax(att_hypothesis, axis=1))
            
        model_outputs[name] =  {
            'Premise': [html_highlight(tokens, attentions, padding_filter='[pad]') for tokens, attentions in zip(tokens_premise, att_premise_rescale)],
            'Hypothesis': [html_highlight(tokens, attentions, padding_filter='[pad]') for tokens, attentions in zip(tokens_hypothesis, att_hypothesis_rescale)],
            'Label': labels,
        #    'Heuristic loss': penalty
        }

    display(HTML(html_render(model_outputs)))
    break

,Premise,Hypothesis,Label
GROUNDTRUTH,This church choir sings to the masses as they sing joyous songs from the book at a church .,The church is filled with song .,entailment
HEURISTIC,This church choir sings to the masses as they sing joyous songs from the book at a church .,The church is filled with song .,entailment
Baseline,This church choir sings to the masses as they sing joyous songs from the book at a church .,The church is filled with song .,entailment
λ=0.01,This church choir sings to the masses as they sing joyous songs from the book at a church .,The church is filled with song .,entailment
λ=0.03,This church choir sings to the masses as they sing joyous songs from the book at a church .,The church is filled with song .,entailment
λ=0.04,This church choir sings to the masses as they sing joyous songs from the book at a church .,The church is filled with song .,entailment
,Premise,Hypothesis,Label
GROUNDTRUTH,"A woman with a green headscarf , blue shirt and a very big grin .",The woman is very happy .,entailment
HEURISTIC,"A woman with a green headscarf , blue shirt and a very big grin .",The woman is very happy .,entailment
Baseline,"A woman with a green headscarf , blue shirt and a very big grin .",The woman is very happy .,entailment


In [105]:
with open('heuristic.html', 'w') as f:
    f.write(html_render(model_outputs))

### La performance en moyenne à partir des logs de tensorboard

In [7]:
import pandas as pd

In [9]:
csv_path = path.join('/Users/dunguyen/Projects/IGRIDA/historic/2022-08-16/regularization_heuristic.csv') # TODO: place it to the cache of lab???

In [10]:
df = pd.read_csv(csv_path)

In [11]:
df

,lamb,hp/accuracy,hp/f1,hp/attention_auc_roc,hp/attention_jaccard,hp/attention_recall
0,0.000,0.796417,0.796417,0.721422,0.514255,0.388206
1,0.005,0.788986,0.788986,0.743282,0.522445,0.448105
2,0.010,0.798046,0.798046,0.742321,0.520490,0.470859
3,0.020,0.794585,0.794585,0.733869,0.513143,0.466456
4,0.030,0.788375,0.788375,0.726778,0.508062,0.454439
5,0.040,0.787765,0.787765,0.723451,0.501418,0.440899
6,0.050,0.789495,0.789495,0.716567,0.495370,0.421600
7,0.060,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.080,0.787358,0.787358,0.706505,0.484290,0.399393
9,0.100,0.785118,0.785118,0.699739,0.480275,0.395884


In [24]:
df.groupby(['lamb'], as_index=False).agg(
    ACC_esp=('hp/accuracy','mean'), ACC_std=('hp/accuracy','std'),
    AUROC_esp=('hp/attention_auc_roc','mean'), AUROC_std=('hp/attention_auc_roc','std'),
    IOU_esp=('hp/attention_jaccard','mean'), IOU_std=('hp/attention_jaccard','std'),
)

,lamb,ACC_esp,ACC_std,AUROC_esp,AUROC_std,IOU_esp,IOU_std
0,0.000,0.794517,0.001722,0.715661,0.007540,0.511966,0.002091
1,0.005,0.394493,0.557897,0.371641,0.525580,0.261223,0.369424
2,0.010,0.795365,0.002989,0.742207,0.000820,0.521552,0.001083
3,0.020,0.793567,0.001200,0.733075,0.000849,0.514063,0.000845
4,0.030,0.789902,0.001835,0.726534,0.000400,0.507579,0.000873
5,0.040,0.787120,0.002854,0.721296,0.001889,0.500353,0.001007
6,0.050,0.789122,0.001117,0.717151,0.000613,0.495207,0.000659
7,0.060,0.000000,NaN,0.000000,NaN,0.000000,NaN
8,0.080,0.787358,NaN,0.706505,NaN,0.484290,NaN
9,0.100,0.785118,NaN,0.699739,NaN,0.480275,NaN


## Statistique sur l'annotation des paires neutres

In [91]:
nb_data = -1
esnli = ESNLIDataset(root=dataset_path, split='test', n_data=nb_data)

In [134]:
from tqdm.notebook import tqdm
import numpy as np

statistic = dict()

for split in ['train', 'test', 'val']:
    esnli = ESNLIDataset(root=dataset_path, split=split, n_data=nb_data)
    data_iter = DataLoader(esnli, batch_size=32, shuffle=False)

    # Comptage par phrase
    hl_p = list()
    hl_h = list()

    for batch in tqdm(data_iter, total=len(data_iter), desc=split):    
        hl_p += ['*' in p for p in batch['highlight_premise']]
        hl_h += ['*' in h for h in batch['highlight_hypothesis']]

    hl_p = np.array(hl_p, dtype=int)
    hl_h = np.array(hl_h, dtype=int)

    statistic[split] = {
        'mutual': hl_p.dot(hl_h),
        'premise only': ((hl_p - hl_h) > 0).sum(),
        'hypothesis only': ((hl_h - hl_p) > 0).sum(),
        'total': len(hl_p)
    }
    
statistic

train:   0%|          | 0/17167 [00:00<?, ?it/s]

test:   0%|          | 0/307 [00:00<?, ?it/s]

val:   0%|          | 0/308 [00:00<?, ?it/s]

{'train': {'mutual': 362630,
  'premise only': 3924,
  'hypothesis only': 182756,
  'total': 549339},
 'test': {'mutual': 6560,
  'premise only': 45,
  'hypothesis only': 3219,
  'total': 9824},
 'val': {'mutual': 6570,
  'premise only': 37,
  'hypothesis only': 3235,
  'total': 9842}}

In [135]:
neutral_statistic = dict()

for split in ['train', 'test', 'val']:
    esnli = ESNLIDataset(root=dataset_path, split=split, n_data=nb_data)
    esnli.data = esnli.data[esnli.data['label'] == 'neutral'].reset_index()
    data_iter = DataLoader(esnli, batch_size=32, shuffle=False)

    # Comptage par phrase
    hl_p = list()
    hl_h = list()

    for batch in tqdm(data_iter, total=len(data_iter), desc=split):    
        hl_p += ['*' in p for p in batch['highlight_premise']]
        hl_h += ['*' in h for h in batch['highlight_hypothesis']]

    hl_p = np.array(hl_p, dtype=int)
    hl_h = np.array(hl_h, dtype=int)

    neutral_statistic[split] = {
        'mutual': hl_p.dot(hl_h),
        'premise only': ((hl_p - hl_h) > 0).sum(),
        'hypothesis only': ((hl_h - hl_p) > 0).sum(),
        'total': len(hl_p)
    }
    
neutral_statistic

train:   0%|          | 0/5712 [00:00<?, ?it/s]

test:   0%|          | 0/101 [00:00<?, ?it/s]

val:   0%|          | 0/102 [00:00<?, ?it/s]

{'train': {'mutual': 0,
  'premise only': 0,
  'hypothesis only': 182756,
  'total': 182756},
 'test': {'mutual': 0,
  'premise only': 0,
  'hypothesis only': 3219,
  'total': 3219},
 'val': {'mutual': 0,
  'premise only': 0,
  'hypothesis only': 3235,
  'total': 3235}}

In [136]:
entail_statistic = dict()

for split in ['train', 'test', 'val']:
    esnli = ESNLIDataset(root=dataset_path, split=split, n_data=nb_data)
    esnli.data = esnli.data[esnli.data['label'] == 'entailment'].reset_index()
    data_iter = DataLoader(esnli, batch_size=32, shuffle=False)

    # Comptage par phrase
    hl_p = list()
    hl_h = list()

    for batch in tqdm(data_iter, total=len(data_iter), desc=split):    
        hl_p += ['*' in p for p in batch['highlight_premise']]
        hl_h += ['*' in h for h in batch['highlight_hypothesis']]

    hl_p = np.array(hl_p, dtype=int)
    hl_h = np.array(hl_h, dtype=int)

    entail_statistic[split] = {
        'mutual': hl_p.dot(hl_h),
        'premise only': ((hl_p - hl_h) > 0).sum(),
        'hypothesis only': ((hl_h - hl_p) > 0).sum(),
        'total': len(hl_p)
    }
    
entail_statistic

train:   0%|          | 0/5732 [00:00<?, ?it/s]

test:   0%|          | 0/106 [00:00<?, ?it/s]

val:   0%|          | 0/105 [00:00<?, ?it/s]

{'train': {'mutual': 179467,
  'premise only': 3924,
  'hypothesis only': 0,
  'total': 183410},
 'test': {'mutual': 3323,
  'premise only': 45,
  'hypothesis only': 0,
  'total': 3368},
 'val': {'mutual': 3292,
  'premise only': 37,
  'hypothesis only': 0,
  'total': 3329}}

In [140]:
contradict_statistic = dict()

for split in ['train', 'test', 'val']:
    esnli = ESNLIDataset(root=dataset_path, split=split, n_data=nb_data)
    esnli.data = esnli.data[esnli.data['label'] == 'contradiction'].reset_index()
    data_iter = DataLoader(esnli, batch_size=32, shuffle=False)

    # Comptage par phrase
    hl_p = list()
    hl_h = list()

    for batch in tqdm(data_iter, total=len(data_iter), desc=split):    
        hl_p += ['*' in p for p in batch['highlight_premise']]
        hl_h += ['*' in h for h in batch['highlight_hypothesis']]

    hl_p = np.array(hl_p, dtype=int)
    hl_h = np.array(hl_h, dtype=int)

    contradict_statistic[split] = {
        'mutual': hl_p.dot(hl_h),
        'premise only': ((hl_p - hl_h) > 0).sum(),
        'hypothesis only': ((hl_h - hl_p) > 0).sum(),
        'total': len(hl_p)
    }
    
contradict_statistic

train:   0%|          | 0/5725 [00:00<?, ?it/s]

test:   0%|          | 0/102 [00:00<?, ?it/s]

val:   0%|          | 0/103 [00:00<?, ?it/s]

{'train': {'mutual': 183163,
  'premise only': 0,
  'hypothesis only': 0,
  'total': 183173},
 'test': {'mutual': 3237,
  'premise only': 0,
  'hypothesis only': 0,
  'total': 3237},
 'val': {'mutual': 3278,
  'premise only': 0,
  'hypothesis only': 0,
  'total': 3278}}